In [ ]:
import sqlite3 as sql
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import nba.s01_data_wrangling as dw
import nba.s02_data_visualisation as dv
import warnings
import os

In [ ]:
DATA_PATH = Path.cwd() / 'data/nba.sqlite'
con = sql.connect(DATA_PATH)


# 1. Insight

In [ ]:
games = dw.select_games(con)
data = dw.create_fg3a_data(games)
regression_results = dw.run_regressions(data)
print(regression_results)

Some markdown explanation here on the visualisation.

In [ ]:
args = {
    'data': regression_results,
    'y': 'fg3a_share_coefficient',
    'x': 'year',
    'se_min': 'se_min',
    'se_max': 'se_max',
    'regression': True,
    'title': 'Importance of 3-Pointers for Winning in the NBA',
    'xlabel': 'Year',
    'ylabel': 'Coefficient (B1) of Share-of-3-Pointer Attempts (fgba3)',
    'linecolor': '#FFC72C'
}
dv.time_series_plot(**args)

# 2. Insight

In [ ]:
# a. Obtain the right data
data = dw.get_avg_fg3_data(con)
print(data)

In [ ]:
# 2. Visualise the data
args = {
    'data': data,
    'x': 'year',
    'y': 'avg_fg3_pct_home',
    'title': 'Success Rate of 3-Point-Shots in the NBA',
    'xlabel': 'Year',
    'regression': True,
    'ylabel': 'Average 3-Point NBA Percentage per Year',
    'linecolor': '#FFC72C'
}
dv.time_series_plot(**args)

Explanatory Text.


# 3. Insight

In [ ]:
# Define the teams for which we want to retrieve the data 
teams = ["Warriors", "Lakers", "76ers", "Suns", "Nuggets", "Mavericks"]

# Add an MA41 column to the data
data = dw.get_ma41_data(con)
data = dw.clean_ma_data(data, teams)

print(data)

In [ ]:
# 2. Visualise the data
args = {
    "data": data,
    "x": "date",
    "y": "ma_41",
    "group_by": "team_name",
    "title": "Avg. Winning Rates of Western Conference Teams",
    "xlabel": "Year",
    "ylabel": "Moving Average (MA-41) of Win Percentage per Team",
    "main_team": "Warriors",
    "linecolor": "#FFC72C",
    "championship_years": [(2015, 2019), (2022, 2022)]
}

dv.multi_team_visualisation(**args)

In [ ]:
con.close()